In [1]:
import rtrtme
import pandas as pd
import json
import numpy as np

In [2]:
data_dir = "data"

In [3]:
client = rtrtme.RTRTMePy()

In [4]:
client.load_config()

In [74]:
event_name = "IRM-PUERTOPRINCESA703-2025"  # "IRM-OMAN703-2025"

In [6]:
event = client.get(f"events/{event_name}")

Registration not required as the token is still valid.
Fetched data from 'events/IRM-PUERTOPRINCESA703-2025'...


In [7]:
event_date = event["date"].replace("-", "")

In [8]:
json.dump(event, open(f"{data_dir}/json/{event_date}_{event_name.lower()}_event.json", "wt"), indent=4, ensure_ascii=False)

In [9]:
pages = client.get_list(f"events/{event_name}/profiles", max=2000)

Registration not required as the token is still valid.
Fetched page 1 of Unknown from 'events/IRM-PUERTOPRINCESA703-2025/profiles'...


In [10]:
json.dump(pages, open(f"{data_dir}/json/{event_date}_{event_name.lower()}_profiles.json", "wt"), indent=4, ensure_ascii=False)

In [11]:
profiles = [profile for page in pages for profile in page["list"]]

In [12]:
results = list()
for i in range(0, len(profiles)):
    print(f"Fetching {i + 1} of {len(profiles)}...")
    try:
        data = client._get(f"events/{event_name}/profiles/{profiles[i]['pid']}/splits", quite=True)
        results.append(data)
    except rtrtme.APIException:
        print(f"No results for {profiles[i]['pid']} with class {profiles[i]['class']}?")

Fetching 1 of 380...
Fetching 2 of 380...
Fetching 3 of 380...
Fetching 4 of 380...
Fetching 5 of 380...
Fetching 6 of 380...
Fetching 7 of 380...
Fetching 8 of 380...
Fetching 9 of 380...
No results for RB3DNMXY with class ag?
Fetching 10 of 380...
Fetching 11 of 380...
Fetching 12 of 380...
Fetching 13 of 380...
Fetching 14 of 380...
Fetching 15 of 380...
Fetching 16 of 380...
No results for RTA26PSJ with class ag?
Fetching 17 of 380...
Fetching 18 of 380...
Fetching 19 of 380...
Fetching 20 of 380...
Fetching 21 of 380...
Fetching 22 of 380...
Fetching 23 of 380...
Fetching 24 of 380...
Fetching 25 of 380...
Fetching 26 of 380...
Fetching 27 of 380...
Fetching 28 of 380...
Fetching 29 of 380...
Fetching 30 of 380...
Fetching 31 of 380...
Fetching 32 of 380...
Fetching 33 of 380...
Fetching 34 of 380...
Fetching 35 of 380...
Fetching 36 of 380...
Fetching 37 of 380...
Fetching 38 of 380...
Fetching 39 of 380...
Fetching 40 of 380...
Fetching 41 of 380...
Fetching 42 of 380...
Fetchin

In [13]:
json.dump(results, open(f"{data_dir}/json/{event_date}_{event_name.lower()}_results.json", "wt"), indent=4)

In [75]:
# Can reload to adjust transformation rather than re-request everything
pages = json.load(open(f"{data_dir}/json/{event_date}_{event_name.lower()}_results.json", "rt"))

In [76]:
df = pd.DataFrame([split for page in pages for result in page for split in result["list"]])

In [77]:
df = df.loc[df["course"] == "tri_sprint", :]
event_name = "IRM-PUERTOPRINCESA-SPRINT-2025"

In [78]:
df.loc[df["point"].str.contains("-703"), "point"] = df["point"].str.replace("-703", "")
df.loc[df["point"].str.endswith("_S"), "point"] = df["point"].str.replace("_S", "")

In [79]:
df["splitTimeSeconds"] = pd.to_timedelta(df["splitTime"])
df["legTimeSeconds"] = pd.to_timedelta(df["legTime"])
df["netTimeSeconds"] = pd.to_timedelta(df["netTime"])
df["waveTimeSeconds"] = pd.to_timedelta(df["waveTime"])

df["splitTimeSeconds"] = df["splitTimeSeconds"].dt.total_seconds()
df["legTimeSeconds"] = df["legTimeSeconds"].dt.total_seconds()
df["netTimeSeconds"] = df["netTimeSeconds"].dt.total_seconds()
df["waveTimeSeconds"] = df["waveTimeSeconds"].dt.total_seconds()

df.loc[df["point"] == "START", "splitTimeSeconds"] = df["splitTimeSeconds"].fillna(0)
df.loc[df["point"] == "START", "legTimeSeconds"] = df["legTimeSeconds"].fillna(0)
df.loc[df["point"] == "START", "netTimeSeconds"] = df["netTimeSeconds"].fillna(0)
df.loc[df["point"] == "START", "waveTimeSeconds"] = df["waveTimeSeconds"].fillna(0)

In [80]:
df = df.drop(columns=["bib_display", "tag", "startTime", "profile_pic", "profile_color", "_ver", "alias", "course", "i", "u", "etnp", "etfp", "results"])

In [81]:
df.loc[df["division"].str.contains("RELAY", na=False, case=False), "divisionType"] = "RELAY"
df.loc[df["division"].str.contains("ODIV", na=False, case=False), "divisionType"] = "OPEN"
df.loc[df["division"].str.contains("PC", na=False, case=False), "divisionType"] = "PC"
df.loc[df["division"].str.contains("GUIDE", na=False, case=False), "divisionType"] = "GUIDE"
df.loc[df["division"].str.contains("PRO", na=False, case=False), "divisionType"] = "PRO"
df["divisionType"] = df["divisionType"].fillna("AG")
#df["divisionType"] = df["divisionType"].fillna("PRO-AG")

In [82]:
df["legDivPos"] = df.loc[df["point"].isin(["SWIM", "T1", "BIKE", "T2", "FINISH"]), :].groupby(by=["point", "divisionType", "division"])["legTimeSeconds"].rank("dense")
df["legSexPos"] = df.loc[df["point"].isin(["SWIM", "T1", "BIKE", "T2", "FINISH"]), :].groupby(by=["point", "divisionType", "sex"])["legTimeSeconds"].rank("dense")
df["legOvrPos"] = df.loc[df["point"].isin(["SWIM", "T1", "BIKE", "T2", "FINISH"]), :].groupby(by=["point", "divisionType"])["legTimeSeconds"].rank("dense")

df["finDivPos"] = df.loc[df["point"] == "FINISH", :].groupby(by=["point", "divisionType", "division"])["netTimeSeconds"].rank("dense")
df["finSexPos"] = df.loc[df["point"] == "FINISH", :].groupby(by=["point", "divisionType", "sex"])["netTimeSeconds"].rank("dense")
df["finOvrPos"] = df.loc[df["point"] == "FINISH", :].groupby(by=["point", "divisionType"])["netTimeSeconds"].rank("dense")

In [83]:
points = df.groupby("point")[["netTimeSeconds"]].min()
points["pointNum"] = points["netTimeSeconds"].rank(method="dense").astype("int64")
points = points.drop(columns=["netTimeSeconds"])

In [84]:
df = df.join(points["pointNum"], on="point")

In [85]:
df["maxPointNum"] = df.groupby(by=["pid"])["pointNum"].transform("max")

In [86]:
df["maxPointNetTimeSeconds"] = df.loc[df["pointNum"] == df["maxPointNum"], "netTimeSeconds"]
df["maxPointNetTimeSeconds"] = df.groupby(by=["pid"])["maxPointNetTimeSeconds"].transform("min")

In [87]:
df = df.sort_values(by=["maxPointNum", "maxPointNetTimeSeconds"], ascending=[False, True])
df["rank"] = np.arange(len(df)) + 1
df["rank"] = df.groupby(by=["maxPointNum", "maxPointNetTimeSeconds"])["rank"].transform("mean")
df["rank"] = df["rank"].rank(method="dense")

In [88]:
df = df.sort_values(by=["rank", "pointNum"])

In [89]:
finishPointNum = df["pointNum"].max()
df["isDNF"] = df["maxPointNum"].values != finishPointNum

In [90]:
df.reset_index(inplace=True, drop=True)

In [91]:
df.to_csv(f"{data_dir}/csv/{event_date}_{event_name.lower()}_tall.csv")

In [92]:
df.loc[df["point"] == "SWIMEND", "point"] = "SWIM"
df.loc[df["point"] == "BIKESTART", "point"] = "T1"
df.loc[df["point"] == "BIKEEND", "point"] = "BIKE"
df.loc[df["point"] == "RUNSTART", "point"] = "T2"

In [93]:
df = df.loc[df["point"].isin(["SWIM", "T1", "BIKE", "T2", "FINISH"]), :]
df["point"] = df["point"].str.lower()
df.loc[:, "finDivPos"] = df.groupby(by=["pid"])["finDivPos"].transform("max")
df.loc[:, "finSexPos"] = df.groupby(by=["pid"])["finSexPos"].transform("max")
df.loc[:, "finOvrPos"] = df.groupby(by=["pid"])["finOvrPos"].transform("max")

In [94]:
df.rename(columns={"legTime": "LegTime", "netTime": "NetTime"}, inplace=True)

In [95]:
# Chunk by athlete to do the pivot
dfs = [y for x, y in df.groupby('pid')]

In [96]:
for i in range(0, len(dfs)):
    dfs[i] = pd.pivot_table(
        dfs[i],
        values=["LegTime", "NetTime"],
        index=["rank", "bib", "name", "country", "sex", "division", "divisionType", "finDivPos", "finSexPos", "finOvrPos", "isDNF"],
        columns="point",
        aggfunc="max",
        dropna=False,
    )

In [97]:
df = pd.concat(dfs)

In [98]:
df = df.swaplevel(0, 1, axis=1).sort_index(axis=1)

In [99]:
df = df.reindex(level="point", columns=["swim", "t1", "bike", "t2", "finish"])

In [100]:
df.reset_index(inplace=True)

In [101]:
# From: https://stackoverflow.com/a/66829689
df = df.pipe(lambda s: s.set_axis(s.columns.map("".join), axis=1))

In [102]:
df = df.sort_values(by=["rank"])

In [103]:
df.reset_index(inplace=True, drop=True)

In [104]:
df.to_csv(f"{data_dir}/csv/{event_date}_{event_name.lower()}_wide.csv")